In [ ]:
import yfinance as yf

RelNS = yf.Ticker("Reliance.NS")

data = RelNS.history(period='1y')

len(data) # Open,High,Low,Close,Volume,Dividends,Stock Splits

In [ ]:
data.to_csv("reliance_price.csv")

In [ ]:
print(RelNS.info)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(data.High, c='g')
plt.plot(data.Close, c='r')

In [ ]:
RelNS.recommendations

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.moneycontrol.com/news/business/stocks/"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

headlines = soup.find_all("h2")

news_list = []

for h in headlines:
    news_list.append(h.text)

news_list


In [ ]:
from moneycontrol import moneycontrol_api as mc

mc.get_news

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
shree = yf.Ticker("SHRIRAMFIN.NS")

shree_data = shree.history('10y')
plt.plot(shree_data.High, c='g')
plt.plot(shree_data.Close, c='r')

In [2]:
import requests
from bs4 import BeautifulSoup, Comment
# Extracting the links of every page for a stock from moneycontrol

HEADERS_PAGES = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/121.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-IN,en;q=0.9",
    "Referer": "https://www.moneycontrol.com/"
}

session = requests.Session()
session.headers.update(HEADERS_PAGES)


mc_base_url = "https://www.moneycontrol.com/news/tags/reliance-industries/news/"

page = 1
all_news = []
seen_links = set()

while True:
    if page == 1:
        url = mc_base_url + "/"
    else:
        url = f"{mc_base_url}/page-{page}/"

    print(f"Scraping page {page}: {url}")

    response = session.get(url, timeout=10)

    if response.status_code != 200:
        print("Stopping — bad response")
        break

    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all("li", class_="clearfix")

    if not articles:
        print("No articles found. Stopping.")
        break

    new_found = False

    for article in articles:
        headline_tag = article.find("h2")
        link_tag = article.find("a", href=True)

        if headline_tag and link_tag:
            link = link_tag["href"]

            if link not in seen_links:
                seen_links.add(link)
                new_found = True

                all_news.append({
                    "headline": headline_tag.text.strip(),
                    "link": link
                })

    if not new_found:
        print("No new articles. Reached last page.")
        break

    page += 1



# Path('Data/raw/moneycontrol/').mkdir(parents=True, exist_ok=True)

Scraping page 1: https://www.moneycontrol.com/news/tags/reliance-industries/news//
Scraping page 2: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-2/
Scraping page 3: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-3/
Scraping page 4: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-4/
Scraping page 5: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-5/
Scraping page 6: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-6/
Scraping page 7: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-7/
Scraping page 8: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-8/
Scraping page 9: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-9/
Scraping page 10: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-10/
Scraping page 11: https://www.moneycontrol.com/news/tags/reliance-industries/news//page-11/
Scraping page

In [3]:
all_news[749]

{'headline': 'Facebook Fuel for India 2020 | Conversation between Mukesh Ambani and Mark Zuckerberg: Read full transcript',
 'link': 'https://www.moneycontrol.com/news/business/facebook-fuel-for-india-2020-conversation-between-mukesh-ambani-and-mark-zuckerberg-read-full-transcript-6230201.html'}

In [4]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import TooManyRedirects
import time

# this is taking permission from the site

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/121.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-IN,en;q=0.9",
    "Referer": "https://www.moneycontrol.com/"
}

session = requests.Session()
session.headers.update(HEADERS)

def fetch_article_soup(url):
    try:
        response = session.get(
            url,
            timeout=10,
            allow_redirects=True
        )
        # print(response.status_code)
        if response.status_code != 200:
            return None

        return BeautifulSoup(response.text, "html.parser")

    except TooManyRedirects:
        print("Redirect loop detected:", url)
        return None

    except requests.RequestException as e:
        print("Request failed:", url, e)
        return None


In [5]:
from datetime import datetime
import pytz
# Extracting time and date data 
# Normalizing it

IST = pytz.timezone("Asia/Kolkata")

def extract_moneycontrol_date(soup):
    schedule_div = soup.find("div", class_="article_schedule")

    if not schedule_div:
        return None, None

    # 1️⃣ Extract date part
    date_span = schedule_div.find("span")
    if not date_span:
        return None, None

    date_part = date_span.text.strip()   # "February 04, 2026"

    # 2️⃣ Extract time part (text after '/')
    full_text = schedule_div.get_text(separator=" ").strip()
    # Example: "February 04, 2026 / 18:17 IST"

    try:
        time_part = full_text.split("/")[-1].strip()  # "18:17 IST"
    except IndexError:
        return None, None

    # 3️⃣ Combine
    raw_date_text = f"{date_part} {time_part}"
    # "February 04, 2026 18:17 IST"

    # 4️⃣ Parse
    try:
        dt = datetime.strptime(
            raw_date_text.replace(" IST", ""),
            "%B %d, %Y %H:%M"
        )
        news_datetime = IST.localize(dt)
    except ValueError:
        return raw_date_text, None

    return raw_date_text, news_datetime


In [6]:
test_url=all_news[50]["link"]
trial_soup = fetch_article_soup(test_url)
print("Success" if soup else "Blocked")

Success


In [7]:
import uuid
import time
from datetime import datetime, timedelta, date
import json
from pathlib import Path
# Structuring the data and saving in json

news_records = []
article_count = 0
''' json structure-
    news_id:
    headline:
    link:
    source:
    raw_date_text:
    news_datetime:
    article_text:
    scraped_at:
    scrape_page:
    status:'''

print("above loop")
missed_urls = []
for news in all_news:
    # print("inside loop - top")
    article_count += 1

    article_url = news["link"]
    headline = news["headline"]
    # print(article_url)
    soup = fetch_article_soup(article_url)
    
    # print(soup)
    if soup is None:
        news_records.append({
            "news_id": str(uuid.uuid4()),
            "headline": headline,
            "link": article_url,
            "source": "moneycontrol",
            "raw_date_text": None,
            "news_datetime": None,
            "article_text":None,
            "scraped_at": datetime.now(IST).isoformat(),
            "scrape_page": None,
            "status": "fetch_failed"
        })
        continue
    # print("mid")
    # 🔹 extract date from article page
    raw_date_text, news_datetime = extract_moneycontrol_date(soup)

    # print(news_datetime.date())
    

    record = {
        "news_id": str(uuid.uuid4()),
        "headline": headline,
        "link": article_url,
        "source": "moneycontrol",
        "raw_date_text": raw_date_text,
        "news_datetime": (
            news_datetime.isoformat()
            if news_datetime else None
        ),
        "article_text":None,
        "scraped_at": datetime.now(IST).isoformat(),
        "scrape_page": None,
        "status": "success" if news_datetime else "date_missing"
    }

    news_records.append(record)

    if article_count % 10 == 0:
        print(f"Processed {article_count} articles")
    
    # print("bottom")

    time.sleep(2)  # polite scraping


Path('Data/raw/moneycontrol/').mkdir(parents=True, exist_ok=True)
with open('Data/raw/moneycontrol/RELIANCE.json','w', encoding='utf-8') as f:
    json.dump(news_records, f, indent=10, ensure_ascii=False)

above loop
Processed 10 articles
Processed 20 articles
Processed 30 articles
Processed 40 articles
Processed 50 articles
Processed 60 articles
Processed 70 articles
Processed 80 articles
Processed 90 articles
Processed 100 articles
Processed 110 articles
Processed 120 articles
Processed 130 articles
Processed 140 articles
Processed 150 articles
Processed 160 articles
Processed 170 articles
Processed 180 articles
Processed 190 articles
Processed 200 articles
Processed 210 articles
Processed 220 articles
Processed 230 articles
Processed 240 articles
Processed 250 articles
Processed 260 articles
Processed 270 articles
Processed 280 articles
Processed 290 articles
Processed 300 articles
Processed 310 articles
Processed 320 articles
Processed 330 articles
Processed 340 articles
Processed 350 articles
Request failed: https://www.moneycontrol.com/news/business/stocks/how-many-fpi-shareholders-do-nifty-50-companies-have%e2%80%8b-10991981.html HTTPSConnectionPool(host='www.moneycontrol.com', po

In [8]:
len(news_records)

750

In [9]:
# converting json to csv file 
import json
import csv

with open('Data/raw/moneycontrol/RELIANCE.json') as file:
    d = json.load(file)

Reliance_df = open('Data/processed/extracted_news/Reliance.csv', "w", newline='')
cw = csv.writer(Reliance_df)
print(d)
c = 0
for data in d:
    if c == 0:
        header = data.keys()
        cw.writerow(header)
        c += 1
    cw.writerow(data.values())

Reliance_df.close()

[{'news_id': '8f730b57-5f0c-408a-844c-e1b5915e1e52', 'headline': "Mukesh Ambani outlines RIL's Rs 10-lakh-crore plans to fuel India's AI growth: Read full speech", 'link': 'https://www.moneycontrol.com/news/india/mukesh-ambani-outlines-ril-s-rs-10-lakh-crore-plans-to-fuel-india-s-ai-growth-read-full-speech-13835793.html', 'source': 'moneycontrol', 'raw_date_text': 'February 19, 2026 14:56 IST', 'news_datetime': '2026-02-19T14:56:00+05:30', 'article_text': None, 'scraped_at': '2026-02-21T12:08:34.601586+05:30', 'scrape_page': None, 'status': 'success'}, {'news_id': '4467618e-a877-4602-9daa-465a46926779', 'headline': 'Buy Reliance Industries; target of Rs 1688: Prabhudas Lilladher', 'link': 'https://www.moneycontrol.com/news/business/stocks/buy-reliance-industries-target-of-rs-1688-prabhudas-lilladher-13833314.html', 'source': 'moneycontrol', 'raw_date_text': 'February 18, 2026 11:13 IST', 'news_datetime': '2026-02-18T11:13:00+05:30', 'article_text': None, 'scraped_at': '2026-02-21T12:08